# LUCAS Dataset imbalanced class prediction problem
## TODO:
- Run Models with different parameters for all different dates in sepparate
- Find way to make random variable transformations
- Test using float features

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA
#from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier#, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC#, SVC
from sklearn.feature_selection import RFE, RFECV#, SelectFromModel, SelectKBest, chi2

from sklearnext.tools import report_model_search_results
from sklearnext.over_sampling import SMOTE, GeometricSMOTE, RandomOverSampler, DensityDistributor
from sklearnext.model_selection import ModelSearchCV
from sklearnext.cluster import KMeans, SOM, AgglomerativeClustering, Birch, SpectralClustering

from imblearn.pipeline import Pipeline

import datetime as dt # used for convenience purposes

%config InlineBackend.figure_format = 'retina'






correlation_threshold = 0.9

In [ ]:
df = pd.read_csv('../data/raw/tabexport2.csv', delimiter=';')

new_columns = {}
for col in df.columns[1:]:
    new_columns[col] = (col.replace('LC08_L1TP_204032_2015', '')[:4]+'_band'+col[-1:])#[1:]
df = df.rename(columns=new_columns)
df.to_csv('../data/interim/data.csv', index=False)

In [ ]:
df.groupby('class').size()

![class labels](https://ec.europa.eu/eurostat/statistics-explained/images/9/99/LUCAS_-_classification_of_land_cover.png)

In [ ]:
# Degree of Dimensionality
DoD = lambda sample_size, features: sample_size/features
ft = len(df.columns)-1 # -1 is used to disregard the column "class"
df.groupby('class').size().apply(lambda sample_size: DoD(sample_size, ft))

In [ ]:
# Imbalance Ratio
IR = lambda majority_class, class_label: majority_class/class_label
majority_class = df.groupby('class').size().max()
df.groupby('class').size().apply(lambda class_size: IR(majority_class, class_size))

In [ ]:
report = pandas_profiling.ProfileReport(df, correlation_threshold=correlation_threshold)
report

In [ ]:
def make_corr_table(df, method='spearman', fig_size=(15,15)):
    corr = df.corr(method=method)
    # remove the upper diagonal of the correlation matrix
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True

    with sns.axes_style("white"):
        fig, ax = plt.subplots()
        fig.set_size_inches(fig_size[0], fig_size[1])
        sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, square=True, ax=ax, cmap='RdBu_r')
    
    
make_corr_table(df)

## Preprocessing stage
- Take highly correlated variable pairs (threshold ρ=0.9)
- Standardize them
- Run PCA on transformed variables
- Keep as many features as necessary such that the minimum explained variability criteria is met

**Note**: Which other features can we extract from this?

Selecting only one of the days and attempt to transform the variables with relatively random calculations

In [ ]:
# Dropping extremely rare classes
#obs_lost_percentage = df[df['class'].isin(['F', 'G', 'H'])].groupby('class').size().sum() / df.groupby('class').size().sum()
#df2 = df[~df['class'].isin(['F', 'G', 'H'])]
#print('Dropped {0:.2f}% of total observations.'.format(obs_lost_percentage*100))

# Selecting a specific day
day_vars = []
for col in df.columns:
    if col.startswith('218'):
        day_vars.append(col)

df2 = df[['class']+day_vars]

X = df2.drop(columns='class')
y = df2['class']

# creating new variables
# (required for logs) min-max standardization
scaler = MinMaxScaler(feature_range=(0.1, 10), copy=True)
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# logs
logs = X.apply(np.log) # logarithm of base e
logs.columns = ['(log_'+i+')' for i in X.columns]
# e^x
exps = X.apply(np.exp) # e^x
exps.columns = ['(exp_'+i+')' for i in X.columns]
# squared variables
sqrd = X.apply(lambda i: i**2)
sqrd.columns = ['('+i+'**2)' for i in X.columns]

X2 = pd.concat([X, logs, exps, sqrd], axis=1)
cols = list(X2.columns)

# sums, multiplications
for i1 in cols:
    for i2 in cols:
        # sums
        X2[f'({i1}+{i2})'] = X2[i1] + X2[i2]
        # multiplications
        X2[f'({i1}*{i2})'] = X2[i1] * X2[i2]
        # divisions
        X2[f'({i1}/{i2})'] = X2[i1] / X2[i2]

X2 = X2.loc[:, (X2!=1).all()].replace([np.inf, -np.inf], np.nan).dropna(axis=1) # 1694 rows × 1670 columns

# Should reject highly correlated variables here
df_corr = X2.corr() #> correlation_threshold
df_corr = pd.DataFrame(np.tril(df_corr), columns=df_corr.columns, index=df_corr.index)
np.fill_diagonal(df_corr.values, 0)

id_0, id_1 = np.where(df_corr>correlation_threshold)
rejected_vars = np.array(df_corr.index)[np.unique(id_0)]

X3 = X2.drop(columns=rejected_vars)

""" One method, although I don't know exactly what's happening here """
#lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X2, y)
#model = SelectFromModel(lsvc, prefit=True)
#X_new = model.transform(X2)
#X_new.shape


""" Univariate testing: perform a chi-square test to the samples to retrieve only the two best features """
#scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
#_X2 = pd.DataFrame(scaler.fit_transform(X2), columns=X2.columns)

#kbest = SelectKBest(chi2, k=48)
#kbest.fit_transform(_X2, y)
#selected_columns = np.array(X2.columns)[kbest.get_support()]

#X_final = X2[selected_columns]

""" Recursive Feature Extraction """
# CV version
lr = LogisticRegression(max_iter=20000, solver='lbfgs', multi_class='auto')
# step may need to be better adjusted, check later
selector = RFECV(lr, step=0.1, min_features_to_select=6, cv=5, verbose=1)
selector = selector.fit(X3, y)

final_cols    = np.array(X3.columns)[selector.ranking_==1]
rejected_cols = np.concatenate([np.array(X3.columns)[selector.ranking_!=1], rejected_vars])

X_final    = X2[final_cols]
X_rejected = X2[rejected_cols]

make_corr_table(X_final)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.1)

# question: Is it a good idea to make additional PCA on other variables with high correlation?
X_train_final    = X_train[final_cols]
X_train_rejected = X_train.loc[:,rejected_cols]

X_test_final    = X_test[final_cols]
X_test_rejected = X_test.loc[:,rejected_cols]

# standardize highly correlated data
sc = StandardScaler()
X_train_rejected[X_train_rejected.columns] = sc.fit_transform(X_train_rejected)
X_test_rejected[X_test_rejected.columns] = sc.transform(X_test_rejected)

In [ ]:
# Running PCA

min_var = 0.95

old_cols = list(X_train_rejected.columns)

# Run PCA on standardized data
pca = PCA()
train_rejected = pca.fit_transform(X_train_rejected)
test_rejected = pca.transform(X_test_rejected)

# % of explained variance for each additional feature
np.cumsum(pca.explained_variance_ratio_)
# Get column name for the principal components' tables
pc_cols  = [f'pc_{i}' for i in range(np.where(np.cumsum(pca.explained_variance_ratio_) > min_var)[0][0])]
# Number of features to keep to get {min_var}% of explained variability
for i in range(np.where(np.cumsum(pca.explained_variance_ratio_) > min_var)[0][0]):
    X_train_rejected[f'pc_{i}'] = train_rejected[:,i]
    X_test_rejected[f'pc_{i}']  = test_rejected[:,i]

# build final dataset
X_train_final = X_train_final.join(X_train_rejected[pc_cols], on=None, how='outer')
X_test_final  = X_test_final.join(X_test_rejected[pc_cols])
make_corr_table(X_train_final)

## Modelling (original model search function)

In [ ]:
def model_search(X, y, approach='standard'):
    """
    Function built for convenience purposes. oversamplers, classifiers etc etc must be edited in the function itself,
    if necessary.
    """
    global oversamplers, classifiers, grid, param_grids, estimators
    
    configs = {
        'scoring': ['f1_weighted', 'geometric_mean_score', 'accuracy'],
        'n_splits': 5,
        'n_runs': 3,
        'random_state': 0,
        'n_jobs': -1,
        'verbose':1
    }
    # original scoring: ['f1_weighted', 'geometric_mean_score', 'accuracy']
    
    oversamplers = [
        ('none', None),
        ('RandomOverSampler', RandomOverSampler()),
        ('smote', SMOTE()),
        ('gsmote', GeometricSMOTE())
    ]

    classifiers = [
        ('GBC', GradientBoostingClassifier()),
        ('DT', DecisionTreeClassifier()),
        ('KNN', KNeighborsClassifier()),
        ('LR', LogisticRegression(solver='lbfgs', penalty='l2', max_iter=1e4)),
        #('RF', RandomForestClassifier()) # not on the G-SMOTE paper
    ]


    grid = {
        'smote': {'k_neighbors': [2, 3, 4, 5]},
        'gsmote': {
            'k_neighbors': [2, 3, 4, 5],
            'truncation_factor': [-1.0, -0.5, .0, 0.25, 0.5, 0.75, 1.0], 
            'deformation_factor': [.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0],
            'selection_strategy': ['combined', 'minority', 'majority']
        },
        'DT':{'max_depth': [3, 6]},
        'KNN':{'n_neighbors':[3,4,5,6,7]},
        'GBC':{
            'max_depth': [3, 6], 
            'n_estimators': [50, 100]
        }
    }

    param_grids = []
    estimators = []
    
    for oversampler in oversamplers:
        for classifier in classifiers:

            # sets up pipeline with name
            name = f'{oversampler[0]}+{classifier[0]}'
            if approach == 'standard':
                estimators.append((name, Pipeline([oversampler, classifier])))
            elif approach == 'onevsrest':
                estimators.append((name, OneVsRestClassifier(Pipeline([oversampler, classifier]))))
            elif approach == 'onevsone':
                estimators.append((name, OneVsOneClassifier(Pipeline([oversampler, classifier]))))
                

            # sets up param grid for the estimator
            param_grid = {}
            if oversampler[0] in grid.keys(): 
                for key, value in grid[oversampler[0]].items():
                    if approach == 'standard':
                        param_grid[f'{name}__{oversampler[0]}__{key}'] = value
                    elif approach in ['onevsrest', 'onevsone']:
                        param_grid[f'{name}__estimator__{oversampler[0]}__{key}'] = value

            if classifier[0]  in grid.keys(): 
                for key, value in grid[classifier[0]].items():
                    if approach == 'standard':
                        param_grid[f'{name}__{classifier[0]}__{key}'] = value
                    elif approach in ['onevsrest', 'onevsone']:
                        param_grid[f'{name}__estimator__{classifier[0]}__{key}'] = value
            if len(param_grid)>0:
                param_grids.append(param_grid)


    model_search_cv = ModelSearchCV(
        estimators=estimators, 
        param_grids=param_grids, 
        scoring=configs['scoring'], 
        cv=StratifiedKFold(n_splits=configs['n_splits'], shuffle=True),
        refit=False, 
        n_jobs=configs['n_jobs'],
        verbose=configs['verbose']
    )

    model_search_cv.fit(X, y)

    return model_search_cv

### Standard Approach

In [ ]:
print('[%s]' % dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
model_search_cv = model_search(X_train_final, y_train, approach='standard')
report_model_search_results(model_search_cv).sort_values('mean_test_f1_weighted', ascending=False)

### One-vs-rest approach

In [ ]:
print('[%s]' % dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
model_search_cv_OVR = model_search(X_train_final, y_train, approach='onevsrest')
report_model_search_results(model_search_cv_OVR).sort_values('mean_test_f1_weighted', ascending=False)

### One-vs-One approach

In [ ]:
print('[%s]' % dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
model_search_cv_OVO = model_search(X_train_final, y_train, approach='onevsone')
report_model_search_results(model_search_cv_OVO).sort_values('mean_test_f1_weighted', ascending=False)

## Modelling part 2
### Applying Normalization

In [ ]:
norms = {'ss':StandardScaler(), 'l1':Normalizer(norm='l1'), 'l2':Normalizer(norm='l2')}
scores = {}

for norm_name, norm_func in norms.items():
    print(f'Testing with {norm_name} normalization.')
    norm_X_train_final = norm_func.fit_transform(X_train_final)
    
    print('[%s] Model searching using standard approach' % dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    norm_model_search_cv = model_search(norm_X_train_final, y_train, approach='standard')
    scores[f'{norm_name}+standard'] = report_model_search_results(norm_model_search_cv).sort_values('mean_test_f1_weighted', ascending=False)
    print('[%s] Model searching using one vs rest approach' % dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    norm_model_search_cv = model_search(norm_X_train_final, y_train, approach='onevsrest')
    scores[f'{norm_name}+onevsrest'] = report_model_search_results(norm_model_search_cv).sort_values('mean_test_f1_weighted', ascending=False)
    print('[%s] Model searching using one vs one approach' % dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    norm_model_search_cv = model_search(norm_X_train_final, y_train, approach='onevsone')
    scores[f'{norm_name}+onevsone'] = report_model_search_results(norm_model_search_cv).sort_values('mean_test_f1_weighted', ascending=False)
    

In [ ]:
for key, df in scores.items():
    print(f'{key}:\t %s' % df['mean_test_f1_weighted'].max())


## Creating proper configurations for running the experiments

It's been observed from the previous experiments that using a mix of **Standard Scaling** and **One-vs-One**/**One-vs-Rest** provide optimal results, which should make it possible to run models as binary experiments **(?)**

### Adapting configurations
https://github.com/IMS-ML-Lab/publications/blob/master/scripts/config.py

In [ ]:
ORIGINAL_CONFIGURATIONS = {
    'oversamplers': [
        ('NO OVERSAMPLING', None),
        ('RANDOM OVERSAMPLING', RandomOverSampler()),
        ('SMOTE', SMOTE(), {'k_neighbors': [3, 5]}),
        ('G-SMOTE', GeometricSMOTE(), {'selection_strategy': ['combined', 'minority', 'majority'], 'k_neighbors': [3, 5], 'truncation_factor': [-1.0, -0.5, .0, 0.25, 0.5, 0.75, 1.0], 'deformation_factor': [.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0]})
    ],
    'classifiers': [
        ('LR', LogisticRegression(solver='lbfgs', max_iter=1e4)),
        ('KNN', KNeighborsClassifier(), {'n_neighbors': [3, 5]}),
        ('DT', DecisionTreeClassifier(), {'max_depth': [3, 6]}),
        ('GBC', GradientBoostingClassifier(), {'max_depth': [3, 6], 'n_estimators': [50, 100]})
    ],
    #'scoring': ['roc_auc', 'f1', 'geometric_mean_score'],
    'scoring': ['f1_weighted', 'geometric_mean_score', 'accuracy'],
    'n_splits': 5,
    'n_runs': 3,
    'random_state': 0
}

NEW_CONFIGURATIONS = {
    'oversamplers': [
        ('NO OVERSAMPLING', None),
        ('RANDOM OVERSAMPLING', RandomOverSampler()),
        ('SMOTE', SMOTE(), {'k_neighbors': [3, 5]}),
        ('G-SMOTE', GeometricSMOTE(), {'selection_strategy': ['combined', 'minority', 'majority'], 'k_neighbors': [3, 5], 'truncation_factor': [-1.0, -0.5, .0, 0.25, 0.5, 0.75, 1.0], 'deformation_factor': [.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0]})
    ],
    'classifiers': [
        ('LR', OneVsOneClassifier(LogisticRegression(solver='lbfgs', max_iter=1e4))),
        ('KNN', OneVsOneClassifier(KNeighborsClassifier()), {'n_neighbors': [3, 5]}),
        ('DT', OneVsOneClassifier(DecisionTreeClassifier()), {'max_depth': [3, 6]}),
        ('GBC', OneVsOneClassifier(GradientBoostingClassifier()), {'max_depth': [3, 6], 'n_estimators': [50, 100]})
    ],
    'scoring': ['roc_auc', 'f1', 'geometric_mean_score'],
    'n_splits': 5,
    'n_runs': 3,
    'random_state': 0
}


from sklearnext.tools import BinaryExperiment

stdscaler = StandardScaler()
norm_X_train_final = stdscaler.fit_transform(X_train_final)

datasets = [
    ('day_', (norm_X_train_final, y_train)),
    
]

n_jobs=-1
verbose=1
name='ovo experiment'

experiment = BinaryExperiment(name, datasets, NEW_CONFIGURATIONS['oversamplers'], NEW_CONFIGURATIONS['classifiers'], NEW_CONFIGURATIONS['scoring'], NEW_CONFIGURATIONS['n_splits'], NEW_CONFIGURATIONS['n_runs'], NEW_CONFIGURATIONS['random_state'])
experiment.run(n_jobs=n_jobs, verbose=verbose)
experiment.summarize_datasets()
experiment.calculate_optimal()
experiment.calculate_wide_optimal()
experiment.calculate_ranking()
experiment.calculate_mean_sem_ranking()
experiment.calculate_mean_sem_scores()
experiment.calculate_mean_sem_perc_diff_scores()
experiment.calculate_friedman_test()
experiment.calculate_holms_test()
experiment.dump(experiment_path)



In [ ]:
def model_search(X, y, configs, approach='standard'):
    """
    Function built for convenience purposes. oversamplers, classifiers etc etc must be edited in the function itself,
    if necessary.
    """
    global oversamplers, classifiers, grid, param_grids, estimators


    model_search_cv = ModelSearchCV(
        estimators=estimators, 
        param_grids=param_grids, 
        scoring=['f1_weighted', 'geometric_mean_score', 'accuracy'], 
        cv=StratifiedKFold(n_splits=5, shuffle=True),
        refit=False, 
        n_jobs=-1,
        verbose=1
    )

    model_search_cv.fit(X, y)

    return model_search_cv

In [ ]:
CONFIGURATIONS['GSMOTE'].keys()


# model name -> ['oversamplers', 'classifiers', 'scoring', 'n_splits', 'n_runs', 'random_state']

# TEMPORARY AREA

In [ ]:
import sqlite3

df = pd.read_csv('../data/interim/data.csv', delimiter=',')
cols = list(df.columns)
cols.remove('class')
cols = cols + ['class']
cols

conn = sqlite3.connect('../data/raw/remote_sensing_data.db')
#query = "SELECT country FROM Population WHERE population > 50000000;"

lucas = pd.read_sql_query('SELECT * FROM lucas', conn)
lucas.columns = cols

# Create sqlite database and cursor
conn = sqlite3.connect('../data/interim/remote_sensing_data.db')
c = conn.cursor()


dfs = {}
days = set([col.split('_band')[0] for col in cols])
bands = [f'band{i}' for i in range(2,8)] + ['class']
days.remove('class')

for day in days:
    day_cols = []
    for col in cols:
        if day in col:
            day_cols.append(col)
    dfs[day] = lucas[day_cols+['class']]
    dfs[day]['day'] = day
    dfs[day] = dfs[day][[list(dfs[day].columns)[-1]] + list(dfs[day].columns)[:-1]]
    dfs[day] = dfs[day].reset_index().rename(columns={'index':'pixel_id'})
    dfs[day].columns = ['pixel_id','day']+bands

ready_df = pd.concat(list(dfs.values())).set_index('pixel_id')
ready_df.to_sql('lucas', conn, if_exists='fail', index=True)


#col_exec = ''
#for col in cols:
#    if col not in ['class', 'day']:
#        col_exec+=f'\n            \'{col}\' NUM,'
#    else:
#        col_exec+=f'\n            \'{col}\' TEXT,'

#col_exec = col_exec[:-1]

#c.execute(f"""CREATE TABLE IF NOT EXISTS lucas (
#            id INTEGER PRIMARY KEY,
#            {col_exec}
#            )""")


#conn.commit()


In [ ]:
ready_df['day'].unique()

In [ ]:
ready_df.shape[0] / 8


In [ ]:
ready_df.dtypes